# Pytorch의 nn.Embedding
- Pytorch의 Embedding Layer는 word2vec과 마찬가지로 word embedding vector를 찾는 **Lookup Table**이다.
    - 단어의 **정수의 고유 index**가 입력으로 들어오면 Embedding Layer의 **그 index의 Vector**를 출력한다.
    - 모델이 학습되는 동안 모델이 풀려는 문제에 맞는 값으로 Embedding Layer의 vector들이 업데이트 된다.
    - Word2Vec의 embedding vector 학습을 nn.Embedding은 자신이 포함된 모델을 학습 하는 과정에서 한다고 생각하면 된다.

In [1]:
import torch
import torch.nn as nn

embd = nn.Embedding(
    num_embeddings=20_000, # vocab size(단어사전의 어휘수) -> 총 몇 개의 단어를 사용할 것인지
    embedding_dim=200, # embedding vector의 차원수 -> 각 단어를 몇 차원으로 표현할 것인지
)

In [3]:
# embd.weight
embd.weight.shape

torch.Size([20000, 200])

In [ ]:
# embedding layer의 입력 - 문서를 구성하는 토큰들의 ID(정수)를 1차원으로 묶어서 전달


# 네이버 영화 댓글 감성분석(Sentiment Analysis)

## 감성분석(Sentiment Analysis) 이란
입력된 텍스트가 **긍적적인 글**인지 **부정적인**인지 또는 **중립적인** 글인지 분석하는 것을 감성(감정) 분석이라고 한다.   
이를 통해 기업이 고객이 자신들의 기업 또는 제품에 대해 어떤 의견을 가지고 있는지 분석한다.

# Dataset, DataLoader 생성

## Korpora에서 Naver 영화 댓글 dataset 가져오기
- https://ko-nlp.github.io/Korpora/ko-docs/corpuslist/nsmc.html
- http://github.com/e9t/nsmc/
    - input: 영화댓글
    - output: 0(부정적댓글), 1(긍정적댓글)
### API
- **corpus 가져오기**
    - `Korpora.load('nsmc')`
- **text/label 조회**
    - `corpus.get_all_texts()` : 전체 corpus의 text들을 tuple로 반환
    - `corpus.get_all_labels()`: 전체 corpus의 label들을 list로 반환
- **train/test set 나눠서 조회**
    - `corpus.train`
    - `corpus.test`
    - `LabeledSentenceKorpusData` 객체에 text와 label들을 담아서 제공.
        - `LabeledSentenceKorpusData.texts`: text들 tuple로 반환.
        - `LabeledSentenceKorpusData.labels`: label들 list로 반환.

## 데이터 로딩

In [1]:
import os
import time

from Korpora import Korpora

corpus = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\Playdata\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\

In [ ]:
all_inputs = corpus.get_all_texts() # X값
all_labels = corpus.get_all_labels() # y값

In [3]:
all_inputs[:5]

('아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다')

In [4]:
all_labels[:5]

[0, 1, 0, 0, 1]

In [5]:
len(all_inputs)

200000

In [6]:
corpus.train

NSMC.train: size=150000
  - NSMC.train.texts : list[str]
  - NSMC.train.labels : list[int]

In [7]:
corpus.train.texts[:5]

('아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다')

## 토큰화
1. 형태소 단위 token화(분절)를 먼저 한다.
    - konlpy로 token화 한 뒤 다시 한 문장으로 만든다.
2. 1에서 처리한 corpus를 BPE 로 token화
   
### 전처리 함수

#### 형태소 단위 분절

In [8]:
from konlpy.tag import Okt
import string
import re

okt = Okt()

# 전처리 = cleansing(필요없는 값 제거) + 정규화(normalize)
def text_preprocessing(text):
    """
    1. 영문 -> 소문자로 변환
    2. 구두점 제거
    3. 형태소 기반 토큰화
    4. 형태소로 토큰화 한 뒤 다시 하나의 문자열로 묶어서 반환.
    """
    text = text.lower()
    # 구두점 제거 (불용어 제거)
    text = re.sub(f"[{string.punctuation}]", " ", text)
    # 정규화
    tokens = okt.morphs(text, stem=True) # stem: 원형 복원
    
    return ' '.join(tokens) # [단어, 단어, 단어, ...]를 "단어 단어 단어 ..."으로 변경해서 반환

In [9]:
print(all_inputs[101]) # 원래 문장
text_preprocessing(all_inputs[101]) # 전처리 후 문장

재미없음 진심 1이훨나 캐스팅두못한듯


'재미없다 진심 1 이훨 나 캐스팅 두 못 한 듯'

In [ ]:
s = time.time()
# train set 전처리
train_texts = corpus.train.texts
train_inputs = [text_preprocessing(txt) for txt in train_texts]
train_labels = corpus.train.labels
# test set 전처리
test_texts = corpus.test.texts
test_inputs = [text_preprocessing(txt) for txt in test_texts]
test_labels = corpus.test.labels

e = time.time()
print(e-s, '초 걸림')

In [16]:
import pickle
import os

os.makedirs("datasets/nsmc", exist_ok=True)
with open("datasets/nsmc/preprocessing_trainset.pkl", "wb") as fw:
    pickle.dump({"input":train_inputs, "output":train_labels}, fw)

In [17]:
with open("datasets/nsmc/preprocessing_testset.pkl", "wb") as fw:
    pickle.dump({"input":test_inputs, "output":test_labels}, fw)

In [ ]:
# pickle로 저장된 전처리된 데이터 불러오기
import pickle
with open("datasets/nsmc/preprocessing_trainset.pkl", "rb") as fr:
    train_dict = pickle.load(fr) # 딕셔너리 형태로 저장되어 있음
with open("datasets/nsmc/preprocessing_testset.pkl", "rb") as fr:
    test_dict = pickle.load(fr)

In [13]:
train_inputs = train_dict["input"]
train_labels = train_dict["output"]
test_inputs = test_dict["input"]
test_labels = test_dict["output"]

In [15]:
all_inputs = train_inputs + test_inputs # vocab 만들때 사용

In [16]:
len(train_inputs), len(test_inputs), len(all_inputs)

(150000, 50000, 200000)

### 토큰화
- Subword 방식 토큰화 적용
- Byte Pair Encoding 방식으로 huggingface tokenizer 사용
    - BPE: 토큰을 글자 단위로 나눈뒤 가장 자주 등장하는 글자 쌍(byte paire)를 찾아 합친뒤 어휘사전에 추가한다.
    - https://huggingface.co/docs/tokenizers/quicktour
    - `pip install tokenizers`

In [17]:
from tokenizers import Tokenizer
from tokenizers.models import BPE # ,Unigram, WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 30_000 # 어휘 사전의 최대 단어 수
min_frequency = 5 # 최소 5번이 나와야 사전에 해당 단어를 추가

tokenizer = Tokenizer(
    BPE(unk_token="[UNK]") # BPE 모델을 사용하여 토큰화할 것임. unk_token은 알 수 없는 단어를 나타내는 토큰
)
tokenizer.pre_tokenizer = Whitespace() # 공백을 기준으로 토큰화할 것임. (띄어쓰기 기준으로 단어를 나눔)
trainer = BpeTrainer( # BPE 트레이너를 사용하여 토큰화 모델을 학습할 것임.
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    special_tokens=["[PAD]", "[UNK]"],
    continuing_subword_prefix="##"
)
tokenizer.train_from_iterator(all_inputs, trainer=trainer) # vocab 생성 (tokenizer 학습)

In [18]:
# 총 vocab size
tokenizer.get_vocab_size()

26738

In [ ]:
# 저장
os.makedirs("saved_models/nsmc", exist_ok=True)
tokenizer.save("saved_models/nsmc/tokenizer_bpe.json")

# load_tokenizer = tokenizer.from_file("saved_models/nsmc/tokenizer_bpe.json")

In [ ]:
idx = 1000
print(all_inputs[idx])
tokens = tokenizer.encode(all_inputs[idx])
print(tokens.ids) # 실제로 사용하는 것은 이거임.
print(tokens.tokens)

정말 최고 의 명작 성인 이 되다 보다 이집트 의 왕자 는 또 다른 감동 그 자체 네 요
[5420, 5438, 2203, 5530, 6570, 2206, 5425, 5410, 14758, 2203, 9123, 923, 1152, 5617, 5450, 651, 5641, 856, 2128]
['정말', '최고', '의', '명작', '성인', '이', '되다', '보다', '이집트', '의', '왕자', '는', '또', '다른', '감동', '그', '자체', '네', '요']


In [20]:
tokenizer.decode(tokens.ids)

'정말 최고 의 명작 성인 이 되다 보다 이집트 의 왕자 는 또 다른 감동 그 자체 네 요'

## Dataset, DataLoader 생성

In [21]:
# dset[0]
tokenizer.encode(train_inputs[0]).ids, train_labels[0]

([1986, 5881, 5426, 5667, 6087], 0)

In [30]:
# Pytorch 사용자 정의 Dataset (Custom Dataset) 정의
# 1. Dataset 상속
# 2. __len__(self): 총 데이터 개수 반환
# 3. __getitem__(self, index): index의 x, y를 반환

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):
    def __init__(self, texts, labels, max_length, tokenizer):
        """
        texts: list - 댓글 목록. 리스트에 댓글들을 담아서 받는다. ["댓글", "댓글", ...]
        labels: list - 댓글 감정 목록. 
        max_length: 개별 댓글의 최대 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다. (Sequence 개수를 맞춘다.)
        tokenizer: Tokenizer
        """
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.lables = labels
        # self.texts: padding이 추가된 토큰 id 리스트
        self.texts = [self.__pad_token_sequences(self.tokenizer.encode(txt).ids) for txt in texts]

    ###########################################################################################
    # id로 구성된 개별 문장 token list를 받아서 패딩 추가 [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
    ############################################################################################
    def __pad_token_sequences(self, token_sequences):
        """
        id로 구성된 개별 문서(댓글)의 token_id list를 받아서 max_length 길이에 맞추는 메소드
        max_length 보다 토큰수가 적으면 [PAD] 추가, 많으면 max_length 크기로 줄인다.
            ex) [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
            ex) [2, 3, 4, 5, 6, 7, 8] => [2, 3, 4, 5, 6] (max_length=5인 경우)
        """
        pad_token_id = self.tokenizer.token_to_id("[PAD]")
        seq_len = len(token_sequences) # 입력받은 토큰 개수
        result = None
        if seq_len > self.max_length:
            result = token_sequences[:self.max_length] # max_length보다 크면 잘라서 반환
        else:
            result = token_sequences + [pad_token_id] * (self.max_length - seq_len)
            
        return result
        
    def __len__(self):
        return len(self.lables) # 총 데이터 개수 반환

    def __getitem__(self, idx):
        """
        idx 번째 text와 label을 학습 가능한 type으로 변환해서 반환
        Parameter
            idx: int 조회할 index
        Return
            tuple: (torch.LongTensor, torch.FloatTensor) - 댓글 토큰_id 리스트, 정답 Label
        """
        txt = self.texts[idx]
        label = self.lables[idx]

        return (torch.tensor(txt, dtype=torch.int64), torch.tensor([label], dtype=torch.float32))
    

In [37]:
# 모든 댓글의 토큰 수 조회
all_input_length = [len(tokenizer.encode(txt).ids) for txt in all_inputs]
all_input_length[:5]

[5, 11, 10, 9, 22]

In [38]:
import numpy as np
np.min(all_input_length), np.max(all_input_length)

(0, 89)

In [39]:
np.quantile(all_input_length, q=[0.9, 0.95])
# 전체 중 90%의 토큰수는 29개 미만, 95%의 토큰수는 41개 미만

array([29., 41.])

In [23]:
MAX_LENGTH = 30
trainset = NSMCDataset(
    train_inputs,
    train_labels,
    max_length=MAX_LENGTH,
    tokenizer=tokenizer
)
testset = NSMCDataset(
    test_inputs,
    test_labels,
    max_length=MAX_LENGTH,
    tokenizer=tokenizer
)

In [24]:
len(trainset), len(testset)

(150000, 50000)

In [25]:
trainset[10]

(tensor([  540, 11354,   506,  2408,  5414,  5426,  2408,  5414,   119,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor(1.))

In [26]:
BATCH_SIZE = 64
train_loader = DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle=True, # 매 epoch마다 데이터 순서를 섞어서 학습
    drop_last=True, # 마지막 배치가 BATCH_SIZE보다 작으면 버림
)
test_loader = DataLoader(
    testset,
    batch_size=BATCH_SIZE,
)

In [27]:
len(train_loader), len(test_loader) # 배치(step)의 개수

(2343, 782)

# 모델링
- Embedding Layer를 이용해 Word Embedding Vector를 추출한다.
- LSTM을 이용해 Feature 추출
- Linear + Sigmoid로 댓글 긍정일 확률 출력
  
![outline](figures/rnn/RNN_outline.png)

## 모델 정의

In [28]:
import torch
import torch.nn as nn
from torchinfo import summary
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [ ]:
# 모델 정의
class NSMCClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, bidirectional=True, dropout_rate=0.2):
        """
        vocab_size: int - 어휘 사전의 단어 개수
        embedding_dim: int - 단어 임베딩 벡터의 차원
        hidden_size: int - LSTM의 은닉 상태 크기(feature 수)
        num_layers: int - LSTM 레이어의 개수
        bidirectional: bool - 양방향 LSTM 사용 여부
        dropout: float - 드롭아웃 비율
        """
        super().__init__()
        # 모델을 구성하는 레이어 정의 - Embedding, LSTM, Dropout, Linear(추론기), Sigmoid
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=tokenizer.token_to_id("[PAD]") # 패딩 토큰의 ID를 지정. padding_idx를 지정하면 해당 토큰은 학습하지 않음.
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim, # 입력 크기: 임베딩 벡터의 차원
            hidden_size=hidden_size, # 은닉 상태 크기
            num_layers=num_layers, # LSTM 레이어 개수
            bidirectional=bidirectional, # 양방향 LSTM 사용 여부
            dropout=dropout_rate if num_layers > 1 else 0 # num_layers가 1보다 크면 드롭아웃 적용
        )
        # LSTM의 출력: out, (hidden, cell)
        # out: 모든 타입 스텝의 hidden state 값
        # hidden: 마지막 타입 스텝의 hidden state 값(단기 기억)
        # cell: 마지막 타입 스텝의 cell state 값(장기 기억)

        input_features = hidden_size * 2 if bidirectional else hidden_size
        self.dropout = nn.Dropout(dropout_rate) # LSTM과 Linear 사이에 과적합 방지를 위해 드롭아웃 레이어 추가
        self.classifier = nn.Linear(input_features , 1) # 출력 1 -> 이진 분류 
        self.sigmoid = nn.Sigmoid() # classifier의 출력값을 0~1 사이로 변환
    def forward(self, X):
        """
        Args:
            X: torch.Tensor - (batch_size, max_length) 크기의 입력 문서 토큰 ID 리스트
        """
        embedding_vectors = self.embedding(X)
        # [batch, seq_len] -> embedding -> [batch, seq_len, embedding_dim]
        embedding_vectors = embedding_vectors.transpose(1, 0) # [batch, seq_len, embedding_dim] -> [seq_len, batch, embedding_dim]
        out, _ = self.lstm(embedding_vectors)
        # out.shape: [seq_len, batch, hidden_size * (2 if bidirectional else 1)]
        # classifier(linear)에는 out의 마지막 index 값을 입력
        output = self.dropout(out[-1]) # 드롭아웃 적용
        output = self.classifier(output)
        last_output = self.sigmoid(output)
        return last_output

## 모델 생성

In [30]:
VOCAB_SIZE = tokenizer.get_vocab_size() # 총 어휘 수
EMBEDDING_DIM = 100 # 임베딩 벡터 차원
HIDDEN_SIZE = 64 # LSTM 은닉 상태 크기
NUM_LAYERS = 2 # LSTM 레이어 개수
BIDIRECTIONAL = True # 양방향 LSTM 사용 여부
DROPOUT_RATE = 0.3 # 드롭아웃 비율

# 모델의 복잡도를 높인다. -> EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS를 크게
# Auto Regressive 모델이 아니면 BIDIRECTIONAL=True로 설정 (양방향 LSTM 사용)

model = NSMCClassifier(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    bidirectional=BIDIRECTIONAL,
    dropout_rate=DROPOUT_RATE
).to(device)
print(model)

NSMCClassifier(
  (embedding): Embedding(26738, 100, padding_idx=0)
  (lstm): LSTM(100, 64, num_layers=2, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (classifier): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [32]:
# summary
i = torch.randint(1, 10, (64, MAX_LENGTH))
# 입력 shape: (batch, seq_len)
summary(model, input_data=i, device=device)
# summary(모델, input_shape) => 내부적으로 입력데이터를 생성해서 추론

Layer (type:depth-idx)                   Output Shape              Param #
NSMCClassifier                           [64, 1]                   --
├─Embedding: 1-1                         [64, 30, 100]             2,673,800
├─LSTM: 1-2                              [30, 64, 128]             184,320
├─Linear: 1-3                            [64, 1]                   129
├─Sigmoid: 1-4                           [64, 1]                   --
Total params: 2,858,249
Trainable params: 2,858,249
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 525.03
Input size (MB): 0.02
Forward/backward pass size (MB): 3.50
Params size (MB): 11.43
Estimated Total Size (MB): 14.95

## 학습

### Train/Test 함수 정의

In [ ]:
# 1 epoch train 하는 함수
def train(model, dataloader, loss_fn, optimizer, device):
    model.train()
    model = model.to(device)
    total_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    # 1 epoch 학습 완료
    return total_loss / len(dataloader) # 1 epoch의 train loss 반환

In [ ]:
def test(model, dataloader, loss_fn, device="cpu"):
    model.eval()
    model = model.to(device)
    total_loss = 0.0
    total_acc = 0.0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred_proba = model(X)
            pred_label = (pred_proba > 0.5).type(torch.int32)
            total_loss += loss_fn(pred_proba, y).item()
            total_acc += (pred_label == y).sum().item()
    # 1 epoch 평가 완료
    return total_loss / len(dataloader), total_acc / len(dataloader.dataset)

### Train

In [45]:
LR = 0.0001
EPOCHS = 3
loss_fn = nn.BCELoss() # 이진 분류이므로 Binary Cross Entropy Loss 사용
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [46]:
import time

train_loss_list = []
val_loss_list = []
val_acc_list = []
s = time.time()
for epoch in range(EPOCHS):
    train_loss = train(model, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = test(model, test_loader, loss_fn, device)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
e = time.time()
print(f"학습 시간: {e-s}초")

Epoch 1/3 - Train Loss: 0.6032, Val Loss: 0.4668, Val Acc: 0.7766
Epoch 2/3 - Train Loss: 0.4263, Val Loss: 0.4117, Val Acc: 0.8104
Epoch 3/3 - Train Loss: 0.3851, Val Loss: 0.3944, Val Acc: 0.8196
학습 시간: 482.1812434196472초


## 모델저장

In [ ]:
torch.save(model.state_dict(), "saved_models/nsmc/model.pt", weight_only=True)

TypeError: save() got an unexpected keyword argument 'weight_only'

# 서비스

## 전처리 함수들

In [51]:
from konlpy.tag import Okt

morph_tokenizer = Okt()
def text_preprocessing(text):
    
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]+", ' ', text)
    return ' '.join(morph_tokenizer.morphs(text, stem=True))

In [52]:
def pad_token_sequences(token_sequences, max_length):
    """padding 처리 메소드."""
    pad_token = tokenizer.token_to_id('[PAD]')  
    seq_length = len(token_sequences)           
    result = None
    if seq_length > max_length:                 
        result = token_sequences[:max_length]
    else:                                            
        result = token_sequences + ([pad_token] * (max_length - seq_length))
    return result

In [54]:
def predict_data_preprocessing(text_list):
    """
    모델에 입력할 수있는 input data를 생성
    Parameter:
        text_list: list - 추론할 댓글리스트
    Return
        torch.LongTensor - 댓글 token_id tensor
    """
    text_list = [text_preprocessing(txt) for txt in text_list]
    token_list = [tokenizer.encode(txt).ids for txt in text_list]
    token_list = [pad_token_sequences(token, MAX_LENGTH) for token in token_list]
    return torch.tensor(token_list, dtype=torch.int64)

## 추론

In [56]:
comment_list = ["아 진짜 재미없다.", "여기 식당 먹을만 해요", "이걸 영화라고 만들었냐?", "기대 안하고 봐서 그런지 괜찮은데.", "이걸 영화라고 만들었나?", "아! 뭐야 진짜.", "재미있는데.", "연기 짱 좋아. 한번 더 볼 의향도 있다.", "뭐 그럭저럭"]
input_tensor = predict_data_preprocessing(comment_list)
input_tensor.shape

torch.Size([9, 30])

In [62]:
def predict(model, comment_list:list[str], input_tensor:torch.tensor, device="cpu"):
    """
    model로 comment_list를 추론해서 긍정/부정적인 댓글인지 출력
    출력 형식
        comment(댓글) label 확률
        "아 재미없다." 부정  0.9
        "너무 재밌다." 긍정  0.8
    """
    model.eval()
    model = model.to(device)
    with torch.no_grad():
        pred = model(input_tensor)
        for txt, pos_proba in zip(comment_list, pred):
            label = "긍정" if pos_proba.item() > 0.5 else "부정"
            proba = pos_proba.item() if pos_proba.item() > 0.5 else 1 - pos_proba.item()
            print(txt, label, round(proba, 3), sep="\t")

In [63]:
predict(model, comment_list, input_tensor, device)

아 진짜 재미없다.	부정	0.982
여기 식당 먹을만 해요	부정	0.699
이걸 영화라고 만들었냐?	부정	0.972
기대 안하고 봐서 그런지 괜찮은데.	긍정	0.645
이걸 영화라고 만들었나?	부정	0.972
아! 뭐야 진짜.	부정	0.926
재미있는데.	긍정	0.969
연기 짱 좋아. 한번 더 볼 의향도 있다.	긍정	0.982
뭐 그럭저럭	부정	0.973


In [68]:
print("분석하려는 댓글을 입력하세요. 종료하려면 !quit 입력")
while True:
    comment = input("댓글:")
    if comment == "!quit":
        print("종료합니다.")
        break
    input_tensor = predict_data_preprocessing([comment])
    predict(model, [comment], input_tensor, device)

분석하려는 댓글을 입력하세요. 종료하려면 !quit 입력
무야호	부정	0.583
무야호	부정	0.583
ㅋㅋ	긍정	0.535
ㅋㅋㅋㅋ	부정	0.527
ㅋㅋㅋㅋㅋㅋㅋㅋ	부정	0.74
ㅋ	부정	0.571
재밌다	긍정	0.97
웃기다	긍정	0.639
흥미롭다	부정	0.609
꽤 볼만하다	긍정	0.926
볼만하다	긍정	0.721
볼만하다	긍정	0.721
멋지다	긍정	0.915
종료합니다.
